<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/Kappa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.metrics import cohen_kappa_score
import pandas as pd
import numpy as np

In [3]:
path1 = "/content/drive/MyDrive/fin/사회_단어_jace.csv"
path2 = "/content/drive/MyDrive/fin/사회_단어_rodri.csv"
rater1 = pd.read_csv(path1)
rater2 = pd.read_csv(path2)
rater1 = rater1.iloc[:, 1]
rater2 = rater2.iloc[:, 1]

In [4]:
valid_indices = ~(rater1.isna() | rater2.isna())
rater1_clean = rater1[valid_indices].astype(float)  # 둘 다 float로 변환, (하나는 소수점, 하나는 정수임)
rater2_clean = rater2[valid_indices].astype(float)

In [5]:
print("rater1 첫 5개 값:", rater1_clean.head().tolist())
print("rater2 첫 5개 값:", rater2_clean.head().tolist())
print("1번 유효 데이터 수:", len(rater1_clean))
print("2번 유효 데이터 수:", len(rater2_clean))

rater1 첫 5개 값: [0.0, 0.0, 0.0, 0.0, 0.0]
rater2 첫 5개 값: [0.0, 0.0, 0.0, 0.0, 0.0]
1번 유효 데이터 수: 2263
2번 유효 데이터 수: 2263


In [6]:
# Cohen's kappa 계산
kappa = cohen_kappa_score(rater1_clean, rater2_clean)
print(f"Cohen's Kappa: {kappa:.4f}")

# 관찰된 일치도 계산
observed_agreement = sum(r1 == r2 for r1, r2 in zip(rater1_clean, rater2_clean)) / len(rater1_clean)
print(f"관찰된 일치도: {observed_agreement:.4f}")

# 고유 값 확인
unique_values = np.unique(np.concatenate([rater1_clean.unique(), rater2_clean.unique()]))
print("고유 값:", unique_values)

# 각 값의 비율 계산
for val in unique_values:
    p_val = sum(rater1_clean == val) / len(rater1_clean)
    q_val = sum(rater2_clean == val) / len(rater2_clean)
    print(f"값 {val}: 평가자1 비율 {p_val:.4f}, 평가자2 비율 {q_val:.4f}")

# 우연에 의한 일치도 계산
chance_agreement = sum(
    sum(rater1_clean == val) / len(rater1_clean) * sum(rater2_clean == val) / len(rater2_clean)
    for val in unique_values
)
print(f"우연에 의한 일치도: {chance_agreement:.4f}")

# 직접 kappa 계산
manual_kappa = (observed_agreement - chance_agreement) / (1 - chance_agreement)
print(f"직접 계산한 Kappa: {manual_kappa:.4f}")

Cohen's Kappa: 0.8603
관찰된 일치도: 0.9655
고유 값: [0. 1.]
값 0.0: 평가자1 비율 0.1379, 평가자2 비율 0.1502
값 1.0: 평가자1 비율 0.8621, 평가자2 비율 0.8498
우연에 의한 일치도: 0.7533
직접 계산한 Kappa: 0.8603


출력값에 대한 설명입니다:

Cohen's Kappa: 0.8603

두 평가자 간의 일치도가 우연 요소를 제외하고 86.03%라는 의미입니다.
0.81-1.00 범위에 있어 '거의 완벽한(almost perfect)' 일치도를 나타냅니다.


관찰된 일치도: 0.9655

두 평가자가 실제로 96.55%의 케이스에서 같은 판단을 내렸습니다.


고유 값: [0. 1.]

두 평가자가 사용한 값은 0과 1뿐입니다(이진분류).


값 0.0: 평가자1 비율 0.1379, 평가자2 비율 0.1502

평가자1은 전체 중 13.79%를 0으로 분류했습니다.
평가자2는 전체 중 15.02%를 0으로 분류했습니다.


값 1.0: 평가자1 비율 0.8621, 평가자2 비율 0.8498

평가자1은 전체 중 86.21%를 1로 분류했습니다.
평가자2는 전체 중 84.98%를 1로 분류했습니다.


우연에 의한 일치도: 0.7533

두 평가자가 무작위로 판단했을 때 우연히 일치할 확률이 75.33%입니다.
계산: (0.1379×0.1502) + (0.8621×0.8498) = 0.7533


직접 계산한 Kappa: 0.8603

공식으로 직접 계산한 kappa 값으로 scikit-learn 결과와 일치합니다.
계산: (0.9655-0.7533)/(1-0.7533) = 0.8603



결론적으로, 두 평가자는 우연의 일치를 고려하더라도 매우 높은 수준의 일치도(0.8603)를 보여주고 있습니다.